<a href="https://colab.research.google.com/github/pv1995/DM-Assignment2/blob/main/DataMiningAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
datapath="https://course.ccs.neu.edu/cs6220/homework-2/data/soc-LiveJournal1Adj.txt" #@param 

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import itertools
import sys
from pyspark.sql import *
# from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=d3dbe401728ba4eacd83f711b0e05ced9784ceb37be03a1a0de2a2b17fdaecf9
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 22 not

In [ ]:
!wget https://course.ccs.neu.edu/cs6220/homework-2/data/soc-LiveJournal1Adj.txt

--2023-03-12 00:07:42--  https://course.ccs.neu.edu/cs6220/homework-2/data/soc-LiveJournal1Adj.txt
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156181 (4.0M) [text/plain]
Saving to: ‘soc-LiveJournal1Adj.txt’

soc-LiveJournal1Adj 100%[===================>]   3.96M  --.-KB/s    in 0.1s    

2023-03-12 00:07:42 (37.8 MB/s) - ‘soc-LiveJournal1Adj.txt’ saved [4156181/4156181]



The function "get_all_friends" parses a line from soc-LiveJournal1Adj.txt into a "friend ownership" structure.
For example, the line: ``0    1,2,3,4`` will get parsed to:
``(0, [1, 2, 3, 4])``
That is, a python tuple where the first element is an int and the second element is a List[int]

In [ ]:
def get_all_friends(line):
    split = line.split()
    user_id = int(split[0])

    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))

    return user_id, friends

The function "get_each_friend" maps the "friend ownership" structure to an array of connections.
For example, the value ``(0, [1, 2, 3])`` will get mapped to:
        [
            ((0,1), 0),
            ((0,2), 0),
            ((0,3), 0),
            ((1,2), 1),
            ((1,3), 1),
            ((2,3), 1)
        ]
The "friend ownership" structure is converted a list of all connection information embedded in the structure. For example, users 0 and 1 are already connected, so that connection is represented by ``((0,1), 0)``, where ``0`` indicates that these users are currently friends.
The structure also encodes the fact that users 1 and 2 have a mutual friend (in the above example, the mutual friend 0). So, the resulting connection is represented by ``((1,2), 1)``, where ``1`` indicates that these users share 1 mutual friend.

In [ ]:
def get_each_friend(f_o):
    user_id = f_o[0]
    friends = f_o[1]

    connections = []

    for friend_id in friends:
        key = (user_id, friend_id)
        if user_id > friend_id:
            key = (friend_id, user_id)

        connections.append(
            (key, 0)
        )

    for friend_pair in itertools.combinations(friends, 2):
        friend_0 = friend_pair[0]
        friend_1 = friend_pair[1]

        key = (friend_0, friend_1)
        if friend_0 > friend_1:
            key = (friend_1, friend_0)
        connections.append(
            (key, 1)
        )

    return connections


The function "count_mutual_friends" counts the total number of mutual friends between two users who are not connected to each other. For example lets say for user Id 1 and 2 there are 21 mutual firends then ``((1, 2), 21)`` encodes that users 1 and 2 share 21 mutual friends. This means that user 1 should be recommended to user 2 AND that user 2 should be recommended to user 1.

In [ ]:
def count_mutual_friends(m):
    connection = m[0]
    count = m[1]

    friend_0 = connection[0]
    friend_1 = connection[1]

    recommendation_0 = (friend_0, (friend_1, count))
    recommendation_1 = (friend_1, (friend_0, count))

    return [recommendation_0, recommendation_1]

The function "sort_recommendations" sorts and truncates the top 10 recommendations based on the total number of mutual firends. The function first sorts by mutual friend count, then by user_id (for equal number of mutual friends between users).

In [ ]:
def sort_recommendations(recs):
    if len(recs) > 1024:

        max_indices = []

        for current_rec_number in range(0, 10):
            current_max_index = 0
            for i in range(1, len(recs)):
                rec = recs[i]
                if rec[1] >= recs[current_max_index][1] and i not in max_indices:
                    current_max_index = i

            max_indices.append(current_max_index)

        recs = [recs[i] for i in max_indices]

    # Sort first by mutual friend count, then by user_id (for equal number of mutual friends between users)
    recs.sort(key=lambda x: (-x[1], x[0]))

    # Map every [(user_id, mutual_count), ...] to [user_id, ...] and truncate to 10 elements
    return list(map(lambda x: x[0], recs))[:10]

Below is the main driver of all functions. The output of recommendations is finally stored in Output.txt

In [ ]:
lines = sc.textFile("soc-LiveJournal1Adj.txt")

all_friends = lines.map(get_all_friends)


# VALUE = 0 indicates that user_id and friend_id are already friends.
# VALUE = 1 indicates that user_id and friend_id are not friends.
friend_edges = all_friends.flatMap(get_each_friend)
friend_edges.cache()


mutual_friend_counts = friend_edges.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .map(lambda edge: (edge[0], sum(edge[1])))


recommendations = mutual_friend_counts.flatMap(count_mutual_friends) \
    .groupByKey() \
    .map(lambda m: (m[0], sort_recommendations(list(m[1]))))

recommendations.collect()

recommendations.saveAsTextFile("Recommendations/Output.txt")
sc.stop()
